In [ ]:
###Detailed refernceing and methodology is written in the submitted paper methods section###
###If there are any questions about the code image analysis please feel free to contact me at my private email: yagmur2986@gmail.com###
###Further Supervision contacts for project specific questions: Sebastian Gilbert and Georgiana Neag###

import tifffile as tiff
import imageio.v3 as iio
import matplotlib.pyplot as plt
import skimage as ski
import napari
import time
import numpy as np
from scipy import ndimage as ndi 
from skimage.exposure import rescale_intensity
from skimage.filters import meijering 
from skimage.morphology import skeletonize_3d
from skimage import data, filters, morphology
from skimage.transform import rescale 
import skimage

In [ ]:
#VISUALISATION FOR NUCLEI

total_start = time.time()

#Import files as tiff for better readability and runtime
print ("reading files...")
start = time.time()
nuclei_original = tiff.imread('[Enter_File]')
nuclei_probability = tiff.imread('[Enter_File]')
print (f"files loaded. Time taken: {time.time() - start: .2f} seconds")


#This allows to check for the dimentions / shape of the files, ensure both are the same,
#if one has an additional parameter e.g. (66, 2, 4202, 1043) it means there are two channels, 
#need to therefore adjust.
print ("Nuclei original shape:", nuclei_original.shape)
print ("Nuclei probability shape:", nuclei_probability.shape)


#complete image normalisation (takes a high amount of computational power) 
print ("normalising files...")
start = time.time()
image_normalisation = rescale_intensity(nuclei_probability, out_range=(0,1))
print(f'Intensity range: [{image_normalisation.min()} - {image_normalisation.max()}]')
print(f'Array type: {image_normalisation.dtype}')
print (f"Normalisation done. Time taken: {time.time() - start: .2f} seconds")

np.save("[Enter_Save_Name]", image_normalisation)

In [ ]:
#Thresholding: Two metjods 
#Otsu's method not suitable for images with uneven illumination or non-uniform background 
#Li's method is, this version will explore li's method 
print ("denoising, thresholding and cleaning...")
start = time.time()
denoised = ndi.median_filter(nuclei_probability, size=3)
thresholded = denoised > filters.threshold_li(denoised)
cleaned = morphology.remove_small_objects(
    morphology.remove_small_holes(thresholded, 2**3),
    20**3)
print (f"denoising, thresholding and cleaning done. Time taken: {time.time() - start: .2f} seconds")

print ("segmenting ..")
start = time.time()
segmented=ndi.label(cleaned)[0]
print (f"Segmenting done. Time taken: {time.time() - start: .2f} seconds")

In [ ]:
#optional checkpoint:
print ("opening napari viewer")
viewer = napari.Viewer()
viewer.add_image(nuclei_original, name='Nuclei Original', colormap='grey')
viewer.add_image(nuclei_probability, name='Nuclei Probability', colormap='viridis')
viewer.add_labels(segmented, name='Segmented nuclei')
viewer.dims.ndisplay = 3
print ("napari viewer opened")

In [ ]:
original_dendrite_voxel = ['Enter Voxels'] #Enter voxel in format e.g. 0.299, 0.241, 0.241
wanted_dendrite_voxel =  #Enter desired voxel for rescaling e.g. 0.241

scale_factors = [ x/wanted_dendrite_voxel for x in original_dendrite_voxel] 

nuclei_rescale = rescale(
    segmented,
    scale=scale_factors,
    order=1,
    preserve_range=True,
    anti_aliasing =True, 
    channel_axis = None)

In [ ]:
print ("Nuclei original segmented shape:", segmented.shape)
print ("Nuclei segmented rescale shape:", nuclei_rescale.shape)

print (segmented.dtype)
print (nuclei_rescale.dtype)

In [ ]:
rescaled_nuclei_int = nuclei_rescale.astype(np.int32)

#Performed standard thresholding on the rescale with: anything not 0 is 1 therfore, binary threshold was performed
BINARY_rescaled_nuclei_rescale = (rescaled_nuclei_int != 0).astype(np.uint8)

tiff.imwrite('[Enter_Save_Name]', BINARY_rescaled_nuclei_rescale)
tiff.imwrite('[Enter_Save_Name]]', rescaled_nuclei_int)
Binary_recale_nuclei_OPEN = tiff.imread('[Enter_File]')

In [ ]:
print (segmented.dtype)
print (nuclei_rescale.dtype)
print (rescaled_nuclei_int.dtype)

In [ ]:
labeled_image, count = skimage.measure.label(Binary_recale_nuclei_OPEN, connectivity=2, return_num=True)
print (count)

In [ ]:
rescaled_nuclei_open = tiff.imread('[Enter_File]')
rescaled_int_nuclei_open = tiff.imread('[Enter_File]')

viewer = napari.Viewer()
viewer.add_image(rescaled_nuclei_open, name='rescaled segmented nuclei', colormap='grey')
viewer.add_image(nuclei_original, name='Nuclei Original', colormap='grey')
viewer.add_image(nuclei_probability, name='Nuclei Probability', colormap='viridis')
viewer.add_labels(segmented, name='Segmented nuclei')
viewer.add_labels(rescaled_int_nuclei_open, name='Segmented rescaled int nuclei')
viewer.add_labels(labeled_image, name='Segmented rescaled BINARY nuclei')
viewer.dims.ndisplay = 3

In [ ]:
np.save("[Enter_Save_Name]", rescaled_nuclei_int)